In [ ]:
#files required for UMIC-seq analysis:
#barcodes.fasta - fasta file with barcodes for demultiplexing
#probe.fasta - sequence directly upstream of UMI
#reference.fasta - reference sequence for calling mutations in consensus cluster sequences

#change path to nanopore output directory

In [ ]:
!guppy_basecaller --input_path ./fast5 --save_path ./fastq --flowcell FLO-MIN106 --kit SQK-LSK109 --device cuda:0

In [132]:
!NanoPlot --drop_outliers --summary sequencing_summary.txt -o ./QC

In [137]:
cat fastq/pass/*.fastq | NanoFilt --length 1750 --maxlength 2000 --quality 8 --summary sequencing_summary.txt > ./filtered.fastq

In [ ]:
pip install tqdm

In [141]:
run UMIC-seq/UMIC-seq_helper_2.py demultiplex --barcodes ./barcodes.fasta --input ./filtered.fastq --threshs 22 29 --output ./demultiplexed

Generating barcode queries...
Loading records...
6324515 entries
Aligning...


100%|██████████| 6324515/6324515 [23:55<00:00, 4405.69it/s]


Assigning...

Sampling some scores (to estimate suitable thresholds):
Fwd scores: [48, 14] Rev scores: [17, 14] Assigned BC: BC02
Fwd scores: [15, 48] Rev scores: [17, 14] Assigned BC: BC03
Fwd scores: [17, 14] Rev scores: [48, 14] Assigned BC: BC02
Fwd scores: [48, 16] Rev scores: [15, 14] Assigned BC: BC02
Fwd scores: [17, 15] Rev scores: [28, 14] Assigned BC: none
Fwd scores: [17, 14] Rev scores: [17, 26] Assigned BC: none
Fwd scores: [48, 14] Rev scores: [17, 16] Assigned BC: BC02
Fwd scores: [17, 14] Rev scores: [17, 36] Assigned BC: BC03
Fwd scores: [17, 16] Rev scores: [27, 18] Assigned BC: none
Fwd scores: [17, 14] Rev scores: [48, 14] Assigned BC: BC02
Fwd scores: [15, 40] Rev scores: [15, 14] Assigned BC: BC03
Fwd scores: [17, 14] Rev scores: [17, 48] Assigned BC: BC03
Fwd scores: [17, 24] Rev scores: [17, 14] Assigned BC: none
Fwd scores: [17, 16] Rev scores: [41, 14] Assigned BC: BC02
Fwd scores: [17, 48] Rev scores: [17, 14] Assigned BC: BC03
Fwd scores: [17, 14] Rev score

Bin 10.0-13.8:	 176868
Bin 13.8-17.6:	 18222796
Bin 17.6-21.4:	 723692
Bin 21.4-25.2:	 124480
Bin 25.2-29.0:	 124376
Bin 29.0-32.8:	 240923
Bin 32.8-36.6:	 374404
Bin 36.6-40.4:	 640534
Bin 40.4-44.2:	 1215050
Bin 44.2-48.0:	 3454937

Thresholds for assignment were set to:
Upper threshold: 29 (highest score must be greater)
Lower threshold: 22 (second highest score must be lower)

Number of sequences assigned to barcodes:
Barcodes: ['BC02', 'BC03', 'none']
Sequence count: [3805752, 2044759, 474004]
Relative (%): [60.17, 32.33, 7.49]

Writing to file...
Finished.


In [145]:
run UMIC-seq/UMIC-seq.py UMIextract --input demultiplexed_BC03.fastq --probe probe.fasta --umi_loc up --umi_len 55 --output ExtractedUMIs_BC03.fasta

2044759 sequences analysed

UMIs extracted: 2005742
Discarded: 1.91%:
Bad alignment: 26318
Incomplete UMI: 12699


In [146]:
run UMIC-seq/UMIC-seq.py UMIextract --input demultiplexed_BC02.fastq --probe probe.fasta --umi_loc up --umi_len 55 --output ExtractedUMIs_BC02.fasta

3805752 sequences analysed

UMIs extracted: 3729062
Discarded: 2.02%:
Bad alignment: 65683
Incomplete UMI: 11007


In [ ]:
run UMIC-seq/UMIC-seq.py clusterfull --input ExtractedUMIs_BC03.fasta --reads demultiplexed_BC03.fastq --aln_thresh 70 --size_thresh 50 --output UMIclusterfull_BC03 --stop_thresh 5

In [ ]:
run UMIC-seq/UMIC-seq.py clusterfull --input ExtractedUMIs_BC02.fasta --reads demultiplexed_BC02.fastq --aln_thresh 70 --size_thresh 50 --output UMIclusterfull_BC02 --stop_thresh 5

In [5]:
!samtools faidx reference.fasta && minimap2 -I 16G -x map-ont --MD -d reference.fasta.mmi reference.fasta

[M::mm_idx_gen::0.003*5.71] collected minimizers
[M::mm_idx_gen::0.004*4.70] sorted minimizers
[M::main::0.011*2.05] loaded/built the index for 1 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.011*2.02] distinct minimizers: 285 (95.79% are singletons); average occurrences: 1.105; average spacing: 5.079; total length: 1600
[M::main] Version: 2.18-r1015
[M::main] CMD: minimap2 -I 16G -x map-ont --MD -d reference.fasta.mmi reference.fasta
[M::main] Real time: 0.012 sec; CPU: 0.024 sec; Peak RSS: 0.040 GB


In [12]:
!cd UMIclusterfull_BC03 && ls *.fasta > clusterfiles_BC03.txt

In [16]:
!cd UMIclusterfull_BC02 && ls *.fasta > clusterfiles_BC02.txt

In [18]:
%%writefile medaka_argument_BC02.txt
mini_align -i UMIclusterfull_BC02/filename -r reference.fasta -p UMIclusterfull_BC02/filename.minialn -m \medaka consensus UMIclusterfull_BC02/filename.minialn.bam UMIclusterfull_BC02/filename.probs --model r941_min_high_g360
medaka variant reference.fasta UMIclusterfull_BC02/filename.probs UMIclusterfull_BC02/filename.vcf
rm UMIclusterfull_BC02/filename.minialn.bam UMIclusterfull_BC02/filename.minialn.bam.bai UMIclusterfull_BC02/filename.probs
mv UMIclusterfull_BC02/filename.vcf UMIclusterfull_BC02/vcf/filename.vcf

Writing medaka_argument_BC02.txt


In [19]:
%%writefile medaka_argument_BC03.txt
mini_align -i UMIclusterfull_BC03/filename -r reference.fasta -p UMIclusterfull_BC03/filename.minialn -m \medaka consensus UMIclusterfull_BC03/filename.minialn.bam UMIclusterfull_BC03/filename.probs --model r941_min_high_g360
medaka variant reference.fasta UMIclusterfull_BC03/filename.probs UMIclusterfull_BC03/filename.vcf
rm UMIclusterfull_BC03/filename.minialn.bam UMIclusterfull_BC03/filename.minialn.bam.bai UMIclusterfull_BC03/filename.probs
mv UMIclusterfull_BC03/filename.vcf UMIclusterfull_BC03/vcf/filename.vcf

Writing medaka_argument_BC03.txt


In [24]:
run UMIC-seq/UMIC-seq_helper_2.py generateSH --input UMIclusterfull_BC02/clusterfiles_BC02.txt --output run_polish_BC02.sh --arguments medaka_argument_BC02.txt --keyword filename

Generating shell script for 10503 files, replacing 'filename'.


In [25]:
run UMIC-seq/UMIC-seq_helper_2.py generateSH --input UMIclusterfull_BC03/clusterfiles_BC03.txt --output run_polish_BC03.sh --arguments medaka_argument_BC03.txt --keyword filename

Generating shell script for 4336 files, replacing 'filename'.


In [ ]:
! ./run_polish_BC02.sh && ./run_polish_BC03.sh

In [59]:
!ls -d UMIclusterfull_BC03/vcf/*.vcf > ./vcffiles_list_bc03.txt

In [60]:
!ls -d UMIclusterfull_BC02/vcf/*.vcf > ./vcffiles_list_bc02.txt

In [63]:
run UMIC-seq/UMIC-seq_helper_2.py vcf2fasta --input vcffiles_list_bc02.txt --min_suppfrac 0.6 --reference reference.fasta --output consensus_sequences_bc02.fasta

Loading vcf filenames and reference sequence
Filtered mutations: ['C365T']cf/cluster_10000.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_10001.fasta.vcf       
Filtered mutations: ['G233A']cf/cluster_10002.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A', 'G1388GT', 'G1454GT']
Filtered mutations: ['G1388GT']/cluster_10006.fasta.vcf       
Filtered mutations: ['C267T', 'G273A', 'G309A', 'C331T', 'C413T', 'G419A', 'A606G', 'T684C', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'G1454GT']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filte

Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['C365T', 'A1356AC']10099.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: []_BC02/vcf/cluster_100.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_10100.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: []_BC02/vcf/cluster_10102.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_10103.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A', 'G1454GT']
Filtered mutations: ['G233A']cf/cluster_10105.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178

Filtered mutations: ['C267T', 'G273A', 'G309A', 'C331T', 'C413T', 'G419A', 'A606G', 'T684C', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A']
Filtered mutations: ['C365T']
Filtered mutations: ['C365T']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'A1356AC', 'C1476T', 'CC1479TT']
Filtered mutations: []
Filtered mutations: ['G234A', 'T260C', 'G273A', 'T290C', 'C293T', 'C300T', 'C305T', 'C561T']
Filtered mutations: []
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A', 'A1356AC']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['G1454GT']
Filtered mutations: []
Filtered mutations: ['C305T']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtere

Filtered mutations: []_BC02/vcf/cluster_10249.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_1024.fasta.vcf       
Filtered mutations: ['G233A']cf/cluster_10250.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_10251.fasta.vcf       
Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: ['A232G', 'T290C', 'C359T', 'A1422AC']    
Filtered mutations: []_BC02/vcf/cluster_10254.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_10255.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_10256.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_10257.fasta.vcf       
Filtered mutations: ['G1454GT']/cluster_10258.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_10259.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_1025.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_10260.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_10261.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T

Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: []_BC02/vcf/cluster_10354.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A', 'A1488AC']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G234A', 'T260C', 'G273A', 'G288A', 'T290C', 'G395A']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: []_BC02/vcf/cluster_10359.fasta.vcf       
Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: []_BC02/vcf/cluster_10360.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_10361.fasta.vcf       
Filtered mutations: ['G233A', 'CC318TT', 'G389A', 'G444A', 'G452A', 'G465A']
Filtered mutations: []_BC02/vcf/cluster_10363.fasta.vcf       
Filtered mutati

...

Filtered mutations: []_BC02/vcf/cluster_9692.fasta.vcf       
Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: []_BC02/vcf/cluster_9695.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_9696.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_9697.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'G1454GT', 'C1476T', 'CC1479TT']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: []_BC02/vcf/cluster_969.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['C267T', 'G273A', 'G309A', 

Filtered mutations: ['A232G', 'T290C', 'C359T']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['C365T']
Filtered mutations: []
Filtered mutations: ['C365T']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'G1388GT', 'C1476T', 'CC1479TT']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G233A', 'G389A', 'G444A', 'G452A', 'G465A']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G233A']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A'

Filtered mutations: ['C365T', 'A1356AC']9847.fasta.vcf       
Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: []_BC02/vcf/cluster_9849.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['A245G', 'G396A', 'C424T', 'A457G', 'G483A']
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['C365T', 'A1356AC']9852.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['A1356AC']/cluster_9854.fasta.vcf       
Filtered mutations: ['C365T']cf/cluster_9855.fasta.vcf       
Filtered mutations: ['C365T', 'A1356AC']9856.fasta.vcf       
Filtered mutations: ['G233A', 'G1388GT']9857.fasta.vcf       
Filtered mutations: ['G233A', 'C267T'

Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'C1476T', 'CC1479TT']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['C267T', 'G273A', 'G309A', 'C331T', 'C413T', 'G419A', 'A606G', 'T684C', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'A1422AC']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['G1454GT']
Filtered mutations: ['C365T']
Filtered mutations: []
Filtered mutations: []
Filtered mutations: ['C267T', 'G273A', 'G309A', 'C331T', 'C413T', 'G419A', 'A606G', 'T684C', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A']
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered mutations: ['C267T', 'G273A', 'G309A', 'C331T', 'C413T', 'G419A'

Filtered mutations: []_BC02/vcf/cluster_9997.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_9998.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'C331T', 'C413T', 'G419A', 'A606G', 'C1127T', 'G1178A', 'G1248A', 'C1278T', 'G1322A', 'G1454GT', 'C1476T', 'CC1479TT']
Filtered mutations: ['C365T']cf/cluster_999.fasta.vcf       
Filtered mutations: []_BC02/vcf/cluster_99.fasta.vcf       
Filtered mutations: ['G233A', 'C267T', 'G309A', 'C331T', 'G419A', 'A606G', 'A1167G', 'G1178A', 'G1261A', 'C1278T', 'G1322A', 'G1333A']
Filtered consensus sequences written to consensus_sequences_bc02.fasta.


In [64]:
run UMIC-seq/UMIC-seq_helper_2.py vcf2fasta --input vcffiles_list_bc03.txt --min_suppfrac 0.6 --reference reference.fasta --output consensus_sequences_bc03.fasta

Loading vcf filenames and reference sequence
Filtered mutations: ['A232G', 'A291G', 'A343G', 'T380C']     
Filtered mutations: []_BC03/vcf/cluster_1001.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1002.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1003.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1004.fasta.vcf       
Filtered mutations: ['A232G', 'G1451C']_1005.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1006.fasta.vcf       
Filtered mutations: ['G1454GT']/cluster_1007.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1008.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1009.fasta.vcf       
Filtered mutations: ['T314C']cf/cluster_100.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1010.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1011.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1012.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1013.fasta.vcf       
Filtered mutations: ['T260

Filtered mutations: []_BC03/vcf/cluster_1149.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_114.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1150.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_1151.fasta.vcf       
Filtered mutations: ['A945G']cf/cluster_1152.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1153.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1154.fasta.vcf       
Filtered mutations: ['G1388GT', 'G1454GT']55.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1156.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1157.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1158.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1159.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_115.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1160.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1161.fasta.vcf       
Filtered mutations: ['T231C']cf/cluster_1162.fasta.vcf       
Filtered m

Filtered mutations: ['A232G']cf/cluster_1303.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1304.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1305.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1306.fasta.vcf       
Filtered mutations: ['A1286AC']/cluster_1307.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1308.fasta.vcf       
Filtered mutations: ['A232G', 'C293G']r_1309.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_130.fasta.vcf       
Filtered mutations: ['A303G']cf/cluster_1310.fasta.vcf       
Filtered mutations: ['A291G', 'C1091A', 'G1197A']a.vcf       
Filtered mutations: ['G1262C']f/cluster_1312.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1313.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1314.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_1315.fasta.vcf       
Filtered mutations: ['A274G']cf/cluster_1316.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1317.fasta.vcf       
Filtered 

Filtered mutations: ['T1524C']f/cluster_1458.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_1459.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_145.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1460.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1461.fasta.vcf       
Filtered mutations: ['T314C', 'A363G']r_1462.fasta.vcf       
Filtered mutations: ['G1454GT']/cluster_1463.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1464.fasta.vcf       
Filtered mutations: ['T1001C']f/cluster_1465.fasta.vcf       
Filtered mutations: ['A303G']cf/cluster_1466.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1467.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1468.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1469.fasta.vcf       
Filtered mutations: ['A261G']cf/cluster_146.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1470.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1471.fasta.vcf       
Filtered m

Filtered mutations: []_BC03/vcf/cluster_1607.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1608.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1609.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_160.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1610.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1611.fasta.vcf       
Filtered mutations: ['A1356AC']/cluster_1612.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1613.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1614.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1615.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1616.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1617.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1618.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_1619.fasta.vcf       
Filtered mutations: ['A232G', 'A486G']r_161.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_1620.fasta.vcf       
Filtered m

...

Filtered mutations: ['A232G', 'A1286AC']4269.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_426.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4270.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4271.fasta.vcf       
Filtered mutations: ['T290C']cf/cluster_4272.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4273.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4274.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4275.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4276.fasta.vcf       
Filtered mutations: ['T260C']cf/cluster_4277.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4278.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4279.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_427.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_4280.fasta.vcf       
Filtered mutations: ['T314C', 'A363G']r_4281.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_4282.fasta.vcf       
Filtered m

Filtered mutations: []_BC03/vcf/cluster_515.fasta.vcf       
Filtered mutations: ['A303G']cf/cluster_516.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_517.fasta.vcf       
Filtered mutations: ['T260C', 'T409C', 'T764C']ta.vcf       
Filtered mutations: ['G1388GT']/cluster_519.fasta.vcf       
Filtered mutations: ['T231C']cf/cluster_51.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_520.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_521.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_522.fasta.vcf       
Filtered mutations: ['A248G']cf/cluster_523.fasta.vcf       
Filtered mutations: ['A579G']cf/cluster_524.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_525.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_526.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_527.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_528.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_529.fasta.vcf       
Filtered mutations: ['A23

Filtered mutations: []_BC03/vcf/cluster_670.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_671.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_672.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_673.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_674.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_675.fasta.vcf       
Filtered mutations: ['A232G']cf/cluster_676.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_677.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_678.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_679.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_67.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_680.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_681.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_682.fasta.vcf       
Filtered mutations: ['A232G', 'A486G']r_683.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_684.fasta.vcf       
Filtered mutations: []_BC

Filtered mutations: []_BC03/vcf/cluster_82.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_830.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_831.fasta.vcf       
Filtered mutations: ['T231C']cf/cluster_832.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_833.fasta.vcf       
Filtered mutations: ['ACC1304A', 'CC1307C', 'GC1309G', 'GCCTCTTCGGGGGTCTCGCGGGGTTTTTTGATCCCTAGCATAA1314G']
Filtered mutations: []_BC03/vcf/cluster_835.fasta.vcf       
Filtered mutations: ['G1388GT']/cluster_836.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_837.fasta.vcf       
Filtered mutations: ['A363G']cf/cluster_838.fasta.vcf       
Filtered mutations: ['G1454GT']/cluster_839.fasta.vcf       
Filtered mutations: ['A1285G']f/cluster_83.fasta.vcf       
Filtered mutations: ['T231C', 'G1454GT']840.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_841.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_842.fasta.vcf       
Filtered mutations: []_BC03/vcf/cluster_8

Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: ['A1356AC']
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered mutations: []
Filtered consensus sequences written to consensus_sequences_bc03.fasta.


In [ ]:
!mkdir vcf_bc02 && mv UMIclusterfull_BC02/vcf vcf_bc02 

In [68]:
!mkdir vcf_bc03 && mv UMIclusterfull_BC03/vcf vcf_bc03

In [ ]:
#OUTPUT
#consesus_sequences_BC02.fasta, consesus_sequences_BC02.fasta - fasta files with all generated consensus sequences
# /vcf_bc02/vcf, /vcf_bc03/vcf - folder with vcf files specifying mutations called in each cluster